In [1]:
# Read in Libraries
from __future__ import division, print_function
from logbook import Logger, StreamHandler
import sys
StreamHandler(sys.stdout).push_application()
log = Logger('Logbook')
import shutil, csv, time
timestr = time.strftime("%Y%m%d")
import ujson as json
import utils; reload(utils)
from utils import *
import gc
# from __future__ import division, print_function
from theano.sandbox import cuda
from vgg16bn import Vgg16BN
from sklearn import metrics
%matplotlib inline

def accuracyfunc(y_act, y_pred):
    return metrics.accuracy_score(np.argmax(y_act, axis=1), np.argmax(y_pred, axis=1))
    
def refresh_directory_structure(name, sub_dirs):
    gdir = os.path.join(path, name)
    if os.path.exists(gdir):
        shutil.rmtree(gdir)
    os.makedirs(gdir)
    for sub_dir in sub_dirs:
        os.makedirs(os.path.join(gdir, sub_dir))

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
# Set Parameters and check files
refresh_directories = False
input_exists = False
full = False
log.info('Set Paramters')
path = "../data/fish/crop/"
batch_size=32
clip = 0.99
bags = 1
load_size = (300,300)#(360, 640)

[2017-03-03 21:57:57.441045] INFO: Logbook: Set Paramters


In [3]:
# Read in our VGG pretrained model
log.info('Get VGG')
model = vgg_ft_bn(8)

# Create our VGG model
log.info('Create VGG')
vgg640 = Vgg16BN(load_size).model
vgg640.pop()
vgg640.input_shape, vgg640.output_shape
vgg640.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

# get labels
(val_classes, trn_classes, val_labels, trn_labels,
    val_filenames, filenames, test_filenames) = get_classes(path)

# Read in filenames
log.info('Read filenames')
raw_filenames = [f.split('/')[-1] for f in filenames]
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]
raw_val_filenames = [f.split('/')[-1] for f in val_filenames]

[2017-03-03 21:57:57.735750] INFO: Logbook: Get VGG
[2017-03-03 21:58:01.006430] INFO: Logbook: Create VGG
Found 2685 images belonging to 8 classes.
Found 622 images belonging to 8 classes.
Found 721 images belonging to 1 classes.
[2017-03-03 21:58:01.839349] INFO: Logbook: Read filenames


In [4]:
log.info('Read in data')
if not input_exists:

    batches = get_batches(path+'train', batch_size=batch_size)
    val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)
    (val_classes, trn_classes, val_labels, trn_labels, 
        val_filenames, filenames, test_filenames) = get_classes(path)
    
    # Fetch our large images 
    # Precompute the output of the convolutional part of VGG
    log.info('Fetch images')
    log.info('Get VGG output')
    log.info('Write VGG output')
    
    test = get_data(path+'test', load_size)
    conv_test_feat = vgg640.predict(test, batch_size=16, verbose=1)
    save_array(path+'../results/conv_test_crop_feat.dat', conv_test_feat)     
    del test, conv_test_feat
    gc.collect()
    
    val = get_data(path+'valid', load_size)
    conv_val_feat = vgg640.predict(val, batch_size=16, verbose=1)
    save_array(path+'../results/conv_val_crop_feat.dat', conv_val_feat)
    del val, conv_val_feat
    gc.collect()
    
    trn = get_data(path+'train', load_size)
    conv_trn_feat = vgg640.predict(trn, batch_size=16, verbose=1)    
    del trn
    gc.collect()
    save_array(path+'../results/conv_trn_crop_feat.dat', conv_trn_feat) 
    del conv_trn_feat
    gc.collect()

    # For memory purposes delete out the original train and validation
    log.info('Clear up memory')
    #del trn, val, test
    gc.collect()
    gc.collect()

[2017-03-03 21:58:01.872001] INFO: Logbook: Read in data
Found 2685 images belonging to 8 classes.
Found 622 images belonging to 8 classes.
Found 2685 images belonging to 8 classes.
Found 622 images belonging to 8 classes.
Found 721 images belonging to 1 classes.
[2017-03-03 21:58:01.907724] INFO: Logbook: Fetch images
[2017-03-03 21:58:01.908376] INFO: Logbook: Get VGG output
[2017-03-03 21:58:01.909034] INFO: Logbook: Write VGG output
Found 721 images belonging to 1 classes.
721/721 [==============================] - 33s    
Found 622 images belonging to 8 classes.
622/622 [==============================] - 29s    
Found 2685 images belonging to 8 classes.
2685/2685 [==============================] - 126s   
[2017-03-03 22:04:49.711088] INFO: Logbook: Clear up memory


In [5]:
gc.collect()
conv_val_feat = load_array(path+'../results/conv_val_crop_feat.dat')
conv_trn_feat = load_array(path+'../results/conv_trn_crop_feat.dat') 
conv_test_feat = load_array(path+'../results/conv_test_crop_feat.dat')

In [12]:
classes = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
def fish_only(mat):
    return np.delete(mat, 4, axis=1)

trn_of_labels = fish_only(trn_labels)
val_of_labels = fish_only(val_labels)

In [13]:
trn_of_labels

(2685, 7)

In [6]:
if full:
    conv_trn_feat = np.concatenate([conv_trn_feat, conv_val_feat])
    trn_labels = np.concatenate([trn_of_labels, val_of_labels]) 
    #trn_bbox = np.concatenate([trn_bbox, val_bbox])
    
# Our Convolutional Net Architecture
log.info('Create and fit CNN')
p=0.6
# Set up the fully convolutional net (FCN); 
conv_layers,_ = split_at(vgg640, Convolution2D)
nf=128; p=0. # No dropout

[2017-03-03 22:04:52.263036] INFO: Logbook: Create and fit CNN


In [7]:
conv_layers[-1].output_shape[1:]

(512, 18, 18)

In [ ]:
gc.collect()
nf = 1024
p  = 0.6
def create_model():
    inp = Input(conv_layers[-1].output_shape[1:])
    x = MaxPooling2D()(inp)
    #x = ZeroPadding2D((1,1))(x)
    #x = Convolution2D(64,3,3, activation='relu', border_mode='same')(x)
    #x = Convolution2D(64,3,3, activation='relu', border_mode='same')(x)
    #x =   Dropout(p/4)(x)
    #x = BatchNormalization(axis=1)(x)
    x = MaxPooling2D()(x)
    x = ZeroPadding2D((1,1))(x)
    x = Convolution2D(512,3,3, activation='relu', border_mode='same')(x)
    #x = ZeroPadding2D((1,1))(x)
    #x = BatchNormalization(axis=1)(x)
    #x = Convolution2D(512,3,3, activation='relu', border_mode='same')(x)
    x =   Dropout(p/4)(x)
    x = BatchNormalization(axis=1)(x)
    x = MaxPooling2D()(x)
    x = ZeroPadding2D((1,1))(x)
    x = Convolution2D(1024,3,3, activation='relu', border_mode='same')(x)
    #x = ZeroPadding2D((1,1))(x)
    #x = BatchNormalization(axis=1)(x)
    #x = Convolution2D(1024,3,3, activation='relu', border_mode='same')(x)
    x =   Dropout(p)(x)
    x = BatchNormalization(axis=1)(x)
    #x1 =   MaxPooling2D()(x)
    x1 =   Convolution2D(7,3,3, border_mode='same')(x)
    x1 =   GlobalAveragePooling2D()(x1)
    #x = BatchNormalization(axis=1)(x)
    #x = Dropout(p)(x)
    #x = Flatten()(x)
    #x = Dense(1024, activation='relu')(x)
    #x = BatchNormalization()(x)
    #x = Dropout(p)(x)
    #x = Dense(512, activation='relu')(x)
    #x = BatchNormalization()(x)
    #x = Dropout(p/2)(x)
    x_class = Dense(7, activation='softmax', name='class')(x1)
    return inp, x_class


model = []
predsls = []
pvalsls = []

for ii in range(10):
    inp, x_class = create_model()
    model.append(Model([inp], [x_class]))
    model[ii].compile(Adam(lr=1e-5, decay=1e-6), loss=['categorical_crossentropy'], metrics=['accuracy'])
    #model[ii].summary()
    model[ii].fit(conv_trn_feat, [trn_of_labels], batch_size=batch_size, nb_epoch=2, 
                 validation_data=(conv_val_feat, [val_of_labels]))
    
    model[ii].optimizer.lr = 1e-5
    model[ii].fit(conv_trn_feat, [trn_of_labels], batch_size=batch_size, nb_epoch=10, 
                 validation_data=(conv_val_feat, [val_of_labels]))

    count = 0
    while count < 1:
        model[ii].fit(conv_trn_feat, [trn_of_labels], batch_size=batch_size, nb_epoch=1, 
                     validation_data=(conv_val_feat, [val_of_labels]))
        predsls.append(model[ii].predict(conv_test_feat, batch_size=batch_size)) # or try 32 batch_size
        pvalsls.append(model[ii].predict(conv_val_feat, batch_size=batch_size))
        val_score = "%.3f" % metrics.log_loss(val_of_labels, sum(pvalsls)/len(pvalsls))
        acc_score = "%.3f" % accuracyfunc(val_of_labels, do_clip(sum(pvalsls)/len(pvalsls), clip))
        log.info('Bagged Validation Logloss ' + str(val_score))
        log.info('Bagged Validation Accuracy ' + str(acc_score))
        count += 1

th
Train on 2685 samples, validate on 622 samples
Epoch 1/2
2685/2685 [==============================] - 9s - loss: 1.5867 - acc: 0.5088 - val_loss: 1.8362 - val_acc: 0.3360
Epoch 2/2
2685/2685 [==============================] - 9s - loss: 1.2002 - acc: 0.6436 - val_loss: 1.5003 - val_acc: 0.5241
Train on 2685 samples, validate on 622 samples
Epoch 1/10
2685/2685 [==============================] - 9s - loss: 1.0135 - acc: 0.6901 - val_loss: 1.3899 - val_acc: 0.5257
Epoch 2/10
2685/2685 [==============================] - 9s - loss: 0.8761 - acc: 0.7292 - val_loss: 1.3343 - val_acc: 0.5338
Epoch 3/10
2685/2685 [==============================] - 9s - loss: 0.7553 - acc: 0.7743 - val_loss: 1.2554 - val_acc: 0.5354
Epoch 4/10
2685/2685 [==============================] - 9s - loss: 0.6565 - acc: 0.8220 - val_loss: 1.2086 - val_acc: 0.5611
Epoch 5/10
2685/2685 [==============================] - 9s - loss: 0.5671 - acc: 0.8574 - val_loss: 1.1821 - val_acc: 0.5788
Epoch 6/10
2685/2685 [========

In [9]:
# metrics.log_loss(val_labels, do_clip(sum(pvalsls)/len(pvalsls), .9999))
preds = sum(predsls)/len(predsls)
subm = do_clip(preds, clip)

if full:
    subm_name = path+'../results/subm_full_crop_of_' + timestr + '.csv' #'.csv.gz'
else:
    subm_name = path+'../results/subm_part_crop_of_' + timestr + '.csv' #'.csv.gz'

classes = ['ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'image', raw_test_filenames)
submission.to_csv(subm_name, index=False)#, compression='gzip')
log.info('Done - files @ ' + subm_name)

ZeroDivisionError: division by zero

In [ ]:
FileLink(subm_name)